# Exploration

In [2]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import mode
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
%matplotlib inline

In [4]:
file_name = 'data/wals-chapter-1.csv'
data = pd.read_csv(file_name)
data.head()

,ID,Language_ID,Language_name,Parameter_ID,Parameter_name,Value,Source,Comment
0,1A-kgi,kgi,Konyagi,1A,Consonant Inventories,Large,Santos-1977,NaN
1,1A-cve,cve,Chuave,1A,Consonant Inventories,Small,Thurman-1970,NaN
2,1A-nbk,nbk,Natügu,1A,Consonant Inventories,Moderately large,Wurm-1972b,NaN
3,1A-ach,ach,Aché,1A,Consonant Inventories,Small,Susnik-1974,NaN
4,1A-aiz,aiz,Aizi,1A,Consonant Inventories,Average,Herault-1971,NaN
